Hi, I'm going to apply 6 supervised machine learning classification models and an ANN model on the given dataset to classify mushrooms as poisonous or eatable.
1. Logistic Regression
2. Support Vector machines (SVC)
3. K-Nearest Neighbours(K-NN)
4. Naive Bayes classifier
5. Decision Tree Classifier
6. Random Forest Classifier
7. Artificial Neural Networks

I'll proceed by converting categorical variables into dummy/indicator variables, then reducing dimensions using Princple Component Analysis to reduce 23 categorical variables (which will become 95 variables after conversion)
to only 2 variables (Principle Components) and training different classification models over these two principle components. Finally, I'll visualize the outputs so that decision boundaries of different models can be seen in 2D-plane. Here the preference is not given to accuracy as the goal is to visualize the decision boundaries. For greater accuracy one can choose more than two variables.

### Importing the Libraries

In [ ]:
import numpy as np 
import pandas as pd
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

### Importing the Dataset

In [ ]:
dataset=pd.read_csv("../input/mushrooms.csv")
dataset.head()

### Checking for nulls

In [ ]:
dataset.isnull().sum()

### Description of Dataset

In [ ]:
dataset.describe()

### Class column is response and rest columns are predictors.
### Seprating Predictors and Response

In [ ]:
X=dataset.drop('class',axis=1) #Predictors
y=dataset['class'] #Response
X.head()

### Encoding categorical data
### Label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
Encoder_X = LabelEncoder() 
for col in X.columns:
    X[col] = Encoder_X.fit_transform(X[col])
Encoder_y=LabelEncoder()
y = Encoder_y.fit_transform(y)

In [ ]:
X.head()

In [ ]:
y

# Poisonous = 1
# Eatable = 0

### Getting dummy variables

In [ ]:
X=pd.get_dummies(X,columns=X.columns,drop_first=True)
X.head()

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Applying PCA with  n_components = 2

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

### Functions to visualize Training & Test Set Results.

In [ ]:
def visualization_train(model):
    sns.set_context(context='notebook',font_scale=2)
    plt.figure(figsize=(16,9))
    from matplotlib.colors import ListedColormap
    X_set, y_set = X_train, y_train
    X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
    plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.6, cmap = ListedColormap(('red', 'green')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(y_set)):
        plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                    c = ListedColormap(('red', 'green'))(i), label = j)
    plt.title("%s Training Set" %(model))
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    plt.legend()
def visualization_test(model):
    sns.set_context(context='notebook',font_scale=2)
    plt.figure(figsize=(16,9))
    from matplotlib.colors import ListedColormap
    X_set, y_set = X_test, y_test
    X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                         np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
    plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
                 alpha = 0.6, cmap = ListedColormap(('red', 'green')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(y_set)):
        plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                    c = ListedColormap(('red', 'green'))(i), label = j)
    plt.title("%s Test Set" %(model))
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    plt.legend()

# Artificial Neural Networks (ANN)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

## Initializing ANN

In [ ]:
classifier = Sequential()

## Adding Layers

In [ ]:
classifier.add(Dense(8, kernel_initializer='uniform', activation= 'relu', input_dim = 2))
classifier.add(Dense(6, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(5, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(4, kernel_initializer='uniform', activation= 'relu'))
classifier.add(Dense(1, kernel_initializer= 'uniform', activation= 'sigmoid'))
classifier.compile(optimizer= 'adam',loss='binary_crossentropy', metrics=['accuracy'])

## Fitting ANN to Training Set

In [ ]:
classifier.fit(X_train,y_train,batch_size=10,epochs=100)

## Predicting the Test Set Results

In [ ]:
y_pred=classifier.predict(X_test)
y_pred=(y_pred>0.5)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test, y_pred))

## Classification Report

In [ ]:
print(classification_report(y_test, y_pred))

## Visualizing ANN Training Set results

In [ ]:
visualization_train(model='ANN')

## Visualizing ANN Test Set results

In [ ]:
visualization_test(model='ANN')

### Creating a func to evaluate model's performance.

In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
def print_score(classifier,X_train,y_train,X_test,y_test,train=True):
    if train == True:
        print("Training results:\n")
        print('Accuracy Score: {0:.4f}\n'.format(accuracy_score(y_train,classifier.predict(X_train))))
        print('Classification Report:\n{}\n'.format(classification_report(y_train,classifier.predict(X_train))))
        print('Confusion Matrix:\n{}\n'.format(confusion_matrix(y_train,classifier.predict(X_train))))
        res = cross_val_score(classifier, X_train, y_train, cv=10, n_jobs=-1, scoring='accuracy')
        print('Average Accuracy:\t{0:.4f}\n'.format(res.mean()))
        print('Standard Deviation:\t{0:.4f}'.format(res.std()))
    elif train == False:
        print("Test results:\n")
        print('Accuracy Score: {0:.4f}\n'.format(accuracy_score(y_test,classifier.predict(X_test))))
        print('Classification Report:\n{}\n'.format(classification_report(y_test,classifier.predict(X_test))))
        print('Confusion Matrix:\n{}\n'.format(confusion_matrix(y_test,classifier.predict(X_test))))

# Logistic Regression Model
## Fitting Logistic Regression model to the Training set

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

classifier.fit(X_train,y_train)

## Logistic Regression Training Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=True)

## Logistic Regression Test Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=False)

## Visualising the Logistic Regression Training set results

In [ ]:
visualization_train('Logistic Reg')

## Visualising the Logistic Regression Test set results

In [ ]:
visualization_test('Logistic Reg')

# Support Vecor (SVC) Classification Model
## Fitting SVC to the Training set

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf',random_state=42)

classifier.fit(X_train,y_train)

## SVC Training Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=True)

## SVC Test Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=False)

## Visualising the SVC Training set results

In [ ]:
visualization_train('SVC')

## Visualising the SVC Test set results

In [ ]:
visualization_test('SVC')

# K Nearest Neighbors (K-NN) Classification Model
## Fitting K-NN to the Training set

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

classifier = KNN()
classifier.fit(X_train,y_train)

## K-NN Training  Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=True)

## K-NN Test Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=False)

## Visualising the K-NN Training set results

In [ ]:
visualization_train('K-NN')

## Visualising the K-NN Test set results

In [ ]:
visualization_test('K-NN')

# Naive Bayes Classification Model
## Fitting Naive Bayes classifier to the Training set

In [ ]:
from sklearn.naive_bayes import GaussianNB as NB

classifier = NB()
classifier.fit(X_train,y_train)

## Naive Bayes Training Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=True)

## Naive Bayes Test Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=False)

## Visualising the Naive Bayes Training set results

In [ ]:
visualization_train('Naive Bayes')

## Visualising the Naive bayes Test set results

In [ ]:
visualization_test('Naive Bayes')

# Decision Tree Classification Model
## Fitting Decision Tree classifier to the Training set

In [ ]:
from sklearn.tree import DecisionTreeClassifier as DT

classifier = DT(criterion='entropy',random_state=42)
classifier.fit(X_train,y_train)

## Decision Tree Training Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=True)

## Decision Tree Test Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=False)

## Visualising the Decision tree Training set results

In [ ]:
visualization_train('Decision Tree')

## Visualising the Decision Tree Test set results

In [ ]:
visualization_test('Decision Tree')

# Random Forest Classification Model
## Fitting Random Forest classifier to the Training set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

## Random Forest Training Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=True)

## Random Forest Test Results

In [ ]:
print_score(classifier,X_train,y_train,X_test,y_test,train=False)

## Visualising the Random Forest Training set results

In [ ]:
visualization_train('Random Forest')

## Visualising the Random Forest Test set results

In [ ]:
visualization_test('Random Forest')

# Results :
| Classifier | Logistic Reg| SVC | K-NN | Naive Bayes | Decision Tree | Random Forest |
| --- | --- | --- | --- | --- | --- | --- |
| Train accuracy score | 0.9057 | 0.9289 | 0.9430 | 0.8980 | 1.0000 | 0.9991 |
| Average accuracy score | 0.9057 | 0.9281 | 0.9314 | 0.8982 | 0.8920 | 0.9288 |
| SD | 0.0097 | 0.0112 | 0.0097 | 0.0114 | 0.0128 | 0.0106 |
| Test accuary score | 0.9028 | 0.9258 | 0.9307 | 0.8966 | 0.9016 | 0.9295 |